<a href="https://colab.research.google.com/github/TaruSora/NLP_100knock/blob/main/100knock_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

90.機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ.

In [1]:
! wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
! tar xvzf kftt-data-1.0.tar.gz

--2022-05-17 03:56:48--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  29.5MB/s    in 3.4s    

2022-05-17 03:56:52 (28.0 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/dat

In [2]:
! pip install -U --quiet ja_ginza
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 59.1 MB 1.1 MB/s 
     |████████████████████████████████| 2.2 MB 59.6 MB/s 
     |████████████████████████████████| 6.0 MB 59.7 MB/s 
     |████████████████████████████████| 457 kB 68.6 MB/s 
     |████████████████████████████████| 653 kB 63.3 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 10.1 MB 59.0 MB/s 
     |████████████████████████████████| 181 kB 78.4 MB/s 
     |████████████████████████████████| 58 kB 8.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
     |████████████████████████████████| 13.9 MB 14.8 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uni

In [3]:
! mkdir -p /content/ginza-data
! cat kftt-data-1.0/data/orig/kyoto-train.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/train.ginza.ja
! cat kftt-data-1.0/data/orig/kyoto-dev.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/dev.ginza.ja
! cat kftt-data-1.0/data/orig/kyoto-test.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/test.ginza.ja

In [4]:
!mkdir -p /content/data
for src, dst in [
    ("/content/ginza-data/train.ginza.ja", "/content/data/train.ja"),
    ("/content/ginza-data/dev.ginza.ja", "/content/data/dev.ja"),
    ("/content/ginza-data/test.ginza.ja", "/content/data/test.ja"),
]:
    with open(src) as f:
        lst = []
        tmp = []
        for x in f:
            x = x.strip()
            if x == 'EOS':
                lst.append(' '.join(tmp))
                tmp = []
            elif x != '':
                tmp.append(x.split('\t')[0])
    with open(dst, 'w') as f:
        for line in lst:
            print(line, file=f)

import re
import spacy

nlp = spacy.load("en_core_web_sm")
for src, dst in [
    ("kftt-data-1.0/data/orig/kyoto-train.en", "/content/data/train.en"),
    ("kftt-data-1.0/data/orig/kyoto-dev.en", "/content/data/dev.en"),
    ("kftt-data-1.0/data/orig/kyoto-test.en", "/content/data/test.en"),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = nlp.make_doc(x)
            x = ' '.join([doc.text for doc in x])
            print(x, file=g)

91.90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

In [5]:
!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

Cloning into 'fairseq'...
remote: Enumerating objects: 31357, done.
remote: Total 31357 (delta 0), reused 0 (delta 0), pack-reused 31357
Receiving objects: 100% (31357/31357), 21.57 MiB | 28.03 MiB/s, done.
Resolving deltas: 100% (23130/23130), done.
/content/fairseq
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 90 kB 8.0 MB/s 
     |████████████████████████████████| 123 kB 57.7 MB/s 
     |████████████████████████████████| 236 kB 70.4 MB/s 
     |████████████████████████████████| 112 kB 61.5 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=f4334bf3b68dda21db474780fcd72ac7334b83c87234b9083d8bfcb127305786
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/3368

In [6]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

In [7]:
!fairseq-preprocess -s ja -t en \
    --trainpref /content/data/train \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

2022-05-17 04:01:51 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-17 04:01:51 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/data', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=

In [8]:
! fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 3 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 91.log \

epoch 001:   0% 0/3618 [00:00<?, ?it/s]/content/fairseq/fairseq/utils.py:375: UserWarning: amp_C fused kernels unavailable, disabling multi_tensor_l2norm; you may get better performance by installing NVIDIA's apex library
  "amp_C fused kernels unavailable, disabling multi_tensor_l2norm; "
epoch 001: 100% 3617/3618 [18:47<00:00,  3.28it/s, loss=9.621, nll_loss=8.709, ppl=418.62, wps=10872.2, ups=3.24, wpb=3352.1, bsz=121.3, num_updates=3600, lr=9.12871e-06, gnorm=2.142, train_wall=31, gb_free=11, wall=1122]
epoch 001 | valid on 'valid' subset:   0% 0/14 [00:00<?, ?it/s]
epoch 001 | valid on 'valid' subset:   7% 1/14 [00:00<00:02,  6.21it/s]
epoch 001 | valid on 'valid' subset:  21% 3/14 [00:00<00:01,  9.28it/s]
epoch 001 | valid on 'valid' subset:  36% 5/14 [00:00<00:00,  9.91it/s]
epoch 001 | valid on 'valid' subset:  50% 7/14 [00:00<00:00, 10.28it/s]
epoch 001 | valid on 'valid' subset:  64% 9/14 [00:00<00:00, 10.54it/s]
epoch 001 | valid on 'valid' subset:  79% 11/14 [00:01<00:00, 1

In [9]:
! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt"

cp: cannot create regular file '/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt': No such file or directory


92.91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．

In [10]:
!fairseq-interactive --path /content/checkpoints/checkpoint_best.pt /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_92.out

2022-05-17 05:02:48 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-17 05:02:50 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

93.91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．

In [11]:
!fairseq-score --sys translate_92.out --ref /content/data/test.en

2022-05-17 05:07:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Namespace(ignore_case=False, order=4, ref='/content/data/test.en', sacrebleu=False, sentence_bleu=False, sys='translate_92.out')
BLEU4 = 2.26, 21.6/3.9/1.0/0.3 (BP=1.000, ratio=1.007, syslen=27838, reflen=27636)


94.91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ

In [1]:
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 1 | grep '^H' | cut -f3 > translate_92_1.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 25 | grep '^H' | cut -f3 > translate_92_25.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 50 | grep '^H' | cut -f3 > translate_92_50.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 75 | grep '^H' | cut -f3 > translate_92_75.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 100 | grep '^H' | cut -f3 > translate_92_100.out

/bin/bash: /content/data/test.ja: No such file or directory
/bin/bash: /content/data/test.ja: No such file or directory
/bin/bash: /content/data/test.ja: No such file or directory
/bin/bash: /content/data/test.ja: No such file or directory
/bin/bash: /content/data/test.ja: No such file or directory


In [2]:
!fairseq-score --sys translate_92_1.out --ref /content/data/test.en > /content/BLEU_score_1
!fairseq-score --sys translate_92_25.out --ref /content/data/test.en > /content/BLEU_score_25
!fairseq-score --sys translate_92_50.out --ref /content/data/test.en > /content/BLEU_score_50
!fairseq-score --sys translate_92_75.out --ref /content/data/test.en > /content/BLEU_score_75
!fairseq-score --sys translate_92_100.out --ref /content/data/test.en > /content/BLEU_score_100

/bin/bash: fairseq-score: command not found
/bin/bash: fairseq-score: command not found
/bin/bash: fairseq-score: command not found
/bin/bash: fairseq-score: command not found
/bin/bash: fairseq-score: command not found


In [3]:
import matplotlib.pyplot as plt
import re

def cal_score(file):
    with open(file) as f:
        line = f.readlines()[1]
        line = re.search(r"(?<=BLEU4 = )\d*\.\d*(?=,)", line)
        return float(line.group())

x = [1, 25, 50, 75, 100]
y = [cal_score("/content/BLEU_score_{}".format(n)) for n in x]
plt.plot(x, y)
plt.xlabel("beam_size")
plt.ylabel("BLEU")
plt.show()

IndexError: ignored

95.トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
import re

spm.SentencePieceTrainer.Train("--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0")

sp = spm.SentencePieceProcessor()
sp.Load("kyoto_ja.model")

for src, dst in [
    ("kftt-data-1.0/data/orig/kyoto-train.ja", "/content/data/train.ja"),
    ("kftt-data-1.0/data/orig/kyoto-dev.ja", "/content/data/dev.ja"),
    ("kftt-data-1.0/data/orig/kyoto-test.ja", "/content/data/test.ja"),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = sp.encode_as_pieces(x)
            x = " ".join(x)
            print(x, file=g)

In [ ]:
!pip install subword-nmt

In [ ]:
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/orig/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-train.en > /content/data/train.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-dev.en > /content/data/dev.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-test.en > /content/data/test.en

In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref /content/data/train \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

In [ ]:
! fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 3 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 95.log \

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt"

In [ ]:
!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_95.out

In [ ]:
!fairseq-score --sys translate_95.out --ref /content/data/test.en

In [ ]:
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 1 | grep '^H' | cut -f3 > translate_95_1.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 25 | grep '^H' | cut -f3 > translate_95_25.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 50 | grep '^H' | cut -f3 > translate_95_50.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 75 | grep '^H' | cut -f3 > translate_95_75.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 100 | grep '^H' | cut -f3 > translate_95_100.out

In [ ]:
!fairseq-score --sys translate_95_1.out --ref /content/data/test.en > /content/BLEU_score_1
!fairseq-score --sys translate_95_25.out --ref /content/data/test.en > /content/BLEU_score_25
!fairseq-score --sys translate_95_50.out --ref /content/data/test.en > /content/BLEU_score_50
!fairseq-score --sys translate_95_75.out --ref /content/data/test.en > /content/BLEU_score_75
!fairseq-score --sys translate_95_100.out --ref /content/data/test.en > /content/BLEU_score_100

In [ ]:
import matplotlib.pyplot as plt
import re

def cal_score(file):
    with open(file) as f:
        line = f.readlines()[1]
        line = re.search(r"(?<=BLEU4 = )\d*\.\d*(?=,)", line)
        return float(line.group())

x = [1, 25, 50, 75, 100]
y = [cal_score("/content/BLEU_score_{}".format(n)) for n in x]
plt.plot(x, y)
plt.xlabel("beam_size")
plt.ylabel("BLEU")
plt.show()

96.Tensorboardなどのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ

In [ ]:
!pip install tensorboardX

In [ ]:
!fairseq-train /content/data --arch transformer\
    --tensorboard-logdir log96 \
    --save-dir save96 \
    --max-epoch 3 \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 3000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4000 > 96.log

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log96

97.ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ．

In [ ]:
!fairseq-train /content/data --arch transformer \
    --save-dir save97 \
    --max-epoch 3 \
    --optimizer adam --clip-norm 1.0 \
    --lr 3e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 6000 > 97.log

In [ ]:
! cp /content/save97/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_3.pt"

In [ ]:
!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_3.pt" /content/data < /content/data/test.ja --beam 10 | grep '^H' | cut -f3 > translate_97.out

In [ ]:
!fairseq-score --sys translate_97.out --ref /content/data/test.en

98.Japanese-English Subtitle Corpus (JESC)やJParaCrawlなどの翻訳データを活用し，KFTTのテストデータの性能向上を試みよ．

In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref "/content/drive/MyDrive/Colab Notebooks/train.jesc" \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

In [ ]:
! fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 1 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 98_1.log \

In [ ]:
! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_4.pt"

In [ ]:
!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_4.pt" /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_98_1.out

In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref /content/data/train \
    --validpref /content/data/dev \
    --destdir /content/data \
    --tgtdict /content/data/dict.en.txt \
    --srcdict /content/data/dict.ja.txt \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

In [ ]:
! fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 2 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 98_2.log \

In [ ]:
! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_5.pt"

In [ ]:
!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_5.pt" /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_98_2.out

In [ ]:
!fairseq-score --sys translate_98_2.out --ref /content/data/test.en

99.ユーザが翻訳したい文を入力すると，その翻訳結果がウェブブラウザ上で表示されるデモシステムを構築せよ．

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

In [ ]:
!pip install fairseq

In [ ]:
from fairseq.models.transformer import TransformerModel

In [ ]:
model = TransformerModel.from_pretrained(
    '/content/checkpoints/',
    checkpoint_file="checkpoint_best.pt",
    data_name_or_path="/content/data",
)

In [ ]:
app = Flask(__name__,template_folder="(前略)/templates")
run_with_ngrok(app)

@app.route("/", methods=["GET"])
def get():
    return render_template("index.html", \
        title = "翻訳", \
        message = "入力してください")

@app.route("/", methods=["POST"])
def post():
    text = request.form["name"]
    return render_template("index.html", \
        title = "翻訳", \
        output = model.translate(text, beam=1)
        message = text+"　　　　　　　→　"+output)

app.run()